# Quality Control

In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger
from IPython.display import clear_output
import spatialdata as sd
import napari

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.widgets.qc_widget import QCWidget

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

2026-02-06 16:24:45.977 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Get a SpatialData object

In [5]:
roi_dir = config.analysis_dir / 'rois'
path_list = [roi_dir / f for f in os.listdir(roi_dir)]
path_list

[WindowsPath('../examples/output/sample_analysis/rois/ROI_000.zarr'),
 WindowsPath('../examples/output/sample_analysis/rois/ROI_001.zarr')]

In [10]:
ind = 1

sdata = sd.read_zarr(path_list[ind])

## Open QC Widget in Napari

In [ ]:
viewer = napari.Viewer()
qc_widget = QCWidget(viewer,sdata)
viewer.window.add_dock_widget(qc_widget, area='right')

In [12]:
sdata

SpatialData object, with associated Zarr store: D:\plex-pipe\examples\output\sample_analysis\rois\ROI_001.zarr
└── Images
      ├── 'CD45': DataTree[cyx] (1, 4864, 4864), (1, 2432, 2432), (1, 1216, 1216)
      ├── 'DAPI': DataTree[cyx] (1, 4864, 4864), (1, 2432, 2432), (1, 1216, 1216)
      └── 'NaKATPase': DataTree[cyx] (1, 4864, 4864), (1, 2432, 2432), (1, 1216, 1216)
with coordinate systems:
    ▸ 'global', with elements:
        CD45 (Images), DAPI (Images), NaKATPase (Images)

In [15]:
%load_ext autoreload
%autoreload 2
from plex_pipe.object_quantification.qc_shape_masker import QcShapeMasker

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
qc_masker = QcShapeMasker(
    table_name = 'instanseg_table',
    qc_prefix = "qc_exclude",
    write_to_disk = True
)

In [18]:
sdata

SpatialData object, with associated Zarr store: C:\BLCA-2_Analysis_todel\cores\Core_000.zarr
├── Images
│     ├── 'CD11C': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'CD44': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'CD45': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'DAPI': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'HES1': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'HLA1': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'NaKATPase': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'pCK26': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     └── 'pS6': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
├── Labels
│     ├── 'blob': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
│     ├── 'cytoplasm': DataTree[yx] (5696, 5568), (2848, 2784), (1424

In [17]:
qc_masker.run(sdata)

2025-11-04 14:24:38.898 | ERROR    | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:37 - Table instanseg_table not present in the spatialdata object.


ValueError: Table instanseg_table not present in the spatialdata object.

In [18]:
from napari_spatialdata import Interactive

In [ ]:
Interactive(sdata)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\napari\utils\_proxies.py:103: FutureWarning: The `_dock_widgets` property is private and should not be used in any plugin code. Please use the `dock_widgets` property instead.
  return super().__getattr__(name)


2025-10-30 15:08:23.550 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-30 15:08:23.970 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


In [ ]:
sdata[self.table_name].layers['test']

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]], shape=(11394, 6))

2025-10-30 12:44:54.194 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-30 12:44:54.203 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-30 12:44:54.586 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


In [44]:
self.sdata['qc_exclude_DAPI'].iloc[2]

geometry    POLYGON ((2196.515 1743.224, 2752.739 1743.224...
Name: 2, dtype: geometry

In [ ]:
s